In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import TensorDataset
from tensorboardX import SummaryWriter

from preprocess import fetch_data
from sklearn.model_selection import train_test_split

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
NAME = "PYTORCH_MODEL"
SAVE_EVERY = 10 # Save every N epochs


INPUT_SIZE = 59
NUM_EPOCHS = 30
BATCH_SIZE = 32
LEARNING_RATE = 1

# Load Data

In [ ]:
X, y = fetch_data(['data/200_fullstate_1v0.log'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_actions, y_train_parameters = np.argmax(y_train[:,:4], axis=1), y_train[:,4:]
y_test_actions, y_test_parameters = np.argmax(y_test[:,:4], axis=1), y_test[:,4:]

In [ ]:
num_train_data = X_train.shape[0]
num_test_data = X_test.shape[0]

In [ ]:
# Dataset
train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train_actions), torch.from_numpy(y_train_parameters))
test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test_actions), torch.from_numpy(y_test_parameters))

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=False)

# Train

In [ ]:
# DASH TURN TACKLE KICK 2 1 1 2

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(in_features=INPUT_SIZE, out_features=1000) 
        self.relu1 = nn.LeakyReLU()
        self.fc2 = nn.Linear(in_features=1000, out_features=512) 
        self.relu2 = nn.LeakyReLU()
        self.fc3 = nn.Linear(in_features=512, out_features=200) 
        self.relu3 = nn.LeakyReLU()
        self.fc4 = nn.Linear(in_features=200, out_features=64)
        self.relu4 = nn.LeakyReLU()
        self.output_actions = nn.Linear(in_features=64, out_features=4)
        
        self.output_parameters_dash = nn.Linear(in_features=64, out_features=2)
        self.output_parameters_turn = nn.Linear(in_features=64, out_features=1)
        self.output_parameters_tackle = nn.Linear(in_features=64, out_features=1)
        self.output_parameters_kick = nn.Linear(in_features=64, out_features=2)
            
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        
        out_actions = self.output_actions(out)
        
        out_dash = self.output_parameters_dash(out)
        out_turn = self.output_parameters_turn(out)
        out_tackle = self.output_parameters_tackle(out)
        out_kick = self.output_parameters_kick(out)
        
        return out_actions, [out_dash, out_turn, out_tackle, out_kick]
    
def get_test_loss(model, criterion_actions, criterion_parameters, test_loader):
    loss = 0
    with torch.no_grad():
        for X, y_actions, y_parameters in test_loader:
            X = X.to(device)
            y_actions = y_actions.to(device)
            y_parameters = y_parameters.to(device)
            outputs_actions, output_parameters_array = model(X)
            loss_actions = criterion_actions(outputs_actions, y_actions)
            
            y_parameters = split_parameters(y_parameters)
            loss_parameters = compute_parameter_loss(output_parameters_array, y_parameters, y_actions, criterion_parameters, p=True) / (2 * N)
            loss = loss_actions + loss_parameters
    return loss.item()        

def compute_parameter_loss(output_parameters_array, y_parameters, y_actions, criterion, p=False):
    N = y_actions.shape[0]
    for i in range(N):
        chosen_action = y_actions[i].item()
        relevant_output_parameters = output_parameters_array[chosen_action][i]
        relevant_y_parameters = y_parameters[chosen_action][i]
        if i == 0:
            loss = criterion(relevant_output_parameters, relevant_y_parameters)
        else:
            loss += criterion(relevant_output_parameters, relevant_y_parameters)
    return loss

def split_parameters(parameters):
    return parameters[:,:2], parameters[:, 2:3], parameters[:, 3:4], parameters[:, 4:]

In [ ]:
model = NeuralNet().to(device).double()
# Loss and optimizer
criterion_actions = nn.CrossEntropyLoss()
criterion_parameters = nn.MSELoss(size_average=False) # MSE Sum
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

writer = SummaryWriter() # Tensorboard

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(NUM_EPOCHS):
    train_loss = 0
    for i, (X, y_actions, y_parameters) in enumerate(train_loader):  
        # Move tensors to the configured device
        X = X.to(device)
        y_actions = y_actions.to(device)
        y_parameters = y_parameters.to(device)
        
        N = X.shape[0] # num of data
        
        # Forward pass
        output_actions, output_parameters_array = model(X)
        loss_actions = criterion_actions(output_actions, y_actions) / N
        
        y_parameters = split_parameters(y_parameters)
        loss_parameters = compute_parameter_loss(output_parameters_array, y_parameters, y_actions, criterion_parameters) / (2 * N)
        
        loss = loss_actions + loss_parameters
        
        train_loss += loss.item()
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    test_loss = get_test_loss(model, criterion_actions, criterion_parameters, test_loader) / num_test_data
    
    writer.add_scalar('loss/1:training_loss', train_loss, epoch + 1)
    writer.add_scalar('loss/2:test_loss', test_loss, epoch + 1)
    
    if (epoch + 1) % SAVE_EVERY == 0:
        filename = './models/%s_%d.model' % (NAME, epoch)
        torch.save(model.state_dict(), filename)
        print("File %s created" % filename)
    
    print('Epoch [{}/{}], Loss: {:.4f}, Test Loss: {:.4f}'.format(epoch+1, NUM_EPOCHS, train_loss, test_loss))